# Prototype Notebook

## Required Inputs
1. Directory of planet images
2. Single LandSat image

## What's applied in this notebook
1. The landsat image is convert from float64 to unit16 & nodata values are set to 0
2. Cloud masking is applied to planet + landsat
3. All the planet images are co-registered via local co-reg to the landsat image
4. The bands in the planet image are re-rodered from BGRNIR to RGBNIR to match the expected model input format
5. The values in the planet tiff are scaled to be 0 - 255 to match the expected model input format
6. The model is applied to the planet image
7. Shorelines are extracted from the segmentation

## Future Upgrades
- Run the good/bad filters on the planet imagery
- Filter out planet images that less than **threshold** size of the ROI
- Filter out planet images by percentage of cloud and cloud shadow coverage
- UPdate workflows to utilize Dask
- Have a function that gets the landsat image with the highest georef and lowest RMSE

In [1]:
import coastseg_planet

## Sample Data for Demo

In [2]:
sample_direc = r'C:\development\coastseg-planet\CoastSeg-Planet\output_zoo'
file_to_seg = r"C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230403_101844_81_247d_3B_AnalyticMS_toar_clip.tif"
landsat_image = r''

# Prepare the LandSat Image 
Convert from float64 to unit16 & nodata values are set to 0

In [3]:
from coastseg_planet.processing import convert_from_float_to_unit16, create_tiled_raster

# Source file path : input landsat image that's cloud free
input_path= r"C:\development\coastseg-planet\CoastSeg-Planet\2023-02-14-10-33-43_L8_ID_mqq51_datetime05-06-24__06_47_51_ms.tif"
# Output file path
landsat_processed_path = r"C:\development\coastseg-planet\CoastSeg-Planet\2023-02-14-10-33-43_L8_ID_mqq51_processed.tif"

landsat_processed_path =convert_from_float_to_unit16(input_path, landsat_processed_path)
landsat_processed_path = create_tiled_raster(input_path, landsat_processed_path)
landsat_processed_path

output_path: C:\development\coastseg-planet\CoastSeg-Planet\2023-02-14-10-33-43_L8_ID_mqq51_processed.tif
Created a new tiled raster file with block size 256x256 at C:\development\coastseg-planet\CoastSeg-Planet\2023-02-14-10-33-43_L8_ID_mqq51_processed.tif


'C:\\development\\coastseg-planet\\CoastSeg-Planet\\2023-02-14-10-33-43_L8_ID_mqq51_processed.tif'

# Apply Cloud Mask to LandSat

In [4]:
# do this later....

# Apply Cloud Masks to Planet
Create a cloud and shadow mask for each image in the directory of planet images

In [5]:
from coastseg_planet.masking import apply_cloudmask_to_dir
# apply cloud mask to directory of planet images and generate cloud masks in that directory for each tif
input_dir = r"C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene"
apply_cloudmask_to_dir(input_dir)

Creating cloud and shadow mask from C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230401_102117_56_2488_3B_udm2_clip.tif
Saving combined mask to C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230401_102117_56_2488_3B_udm2_clip_combined_mask_bool.tif
Saving combined mask to C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230401_102117_56_2488_3B_udm2_clip_combined_mask.tif
Creating cloud and shadow mask from C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230401_102119_76_2488_3B_udm2_clip.tif
Saving combined mask to C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230401_102119_76_2488_3B_udm2_clip_combined_mask_bool.tif
Saving combined mask to C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b

# Coregister all the planet images to the landsat image
All the planet images are co-registered via local co-reg to the landsat image

In [6]:
from coastseg_planet.coregister import local_coregister_gw,get_cpus

# try to coregister two planet images together that have clouds now that we can mask out clouds

REFERENCE = r"C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230403_101844_81_247d_3B_AnalyticMS_toar_clip.tif"
INFILE = r"C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230412_094432_88_2439_3B_AnalyticMS_toar_clip.tif"
TARGET_CLOUD_MASK = r"C:\development\coastseg-planet\floris_roi_TOAR_enabled\d1c0c095-a00d-42f8-b6c5-7dbf1f21d3b6\PSScene\20230412_094432_88_2439_3B_udm2_clip_combined_mask.tif"

outfile=r"C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregister_with_combined_mask.tif"

num_cpus = get_cpus()
coregistered_planet_path = local_coregister_gw(REFERENCE, INFILE, outfile, target_cloud_mask = TARGET_CLOUD_MASK,num_cpus=num_cpus) 
# this isn't working and I'll need to figure out why
# coregistered_planet_path = local_coregister_gw(landsat_processed_path, INFILE, outfile, target_cloud_mask = TARGET_CLOUD_MASK,num_cpus=num_cpus) 

coregistered_planet_path


Number of CPUs available: 20


Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00
Polygonize progress     |--------------------------------------------------| 0.0% Complete  => 0:00:00

Calculating footprint polygon and actual data corner coordinates for reference image...
Bounding box of calculated footprint for reference image:
	(662619.0, 5922558.0, 667092.0, 5927034.0)
Calculating footprint polygon and actual data corner coordinates for image to be shifted...


Polygonize progress     |==================================================| 100.0% Complete  => 0:00:00


Bounding box of calculated footprint for image to be shifted:
	(662619.0, 5922558.0, 667092.0, 5927034.0)
Matching window position (X,Y): 664855.4990364996/5924794.563420993
No clear match found yet. Jumping to iteration 2...
input shifts:  -41 -3
No clear match found yet. Jumping to iteration 3...
input shifts:  0 59
Detected integer shifts (X/Y):                            0/0
Detected subpixel shifts (X/Y):                           0.10465466820119154/0.3203018290162849
Calculated total shifts in fft pixel units (X/Y):         0.10465466820119154/0.3203018290162849
Calculated total shifts in reference pixel units (X/Y):   0.10465466820119154/0.3203018290162849
Calculated total shifts in target pixel units (X/Y):      0.10465466820119154/0.3203018290162849
Calculated map shifts (X,Y):				  0.31396400462836027/-0.9609054867178202
Calculated absolute shift vector length in map units:     1.0108970029665179
Calculated angle of shift vector in degrees from North:   341.9058356051124
Ori

09:44:55:WARNING:755:io.save:The file C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregister_with_combined_mask.tif already exists.


<xarray.DataArray 'array-1d77f6bd1eb563e5e7eb04fec63aad37' (band: 4, y: 1492,
                                                            x: 1491)> Size: 71MB
dask.array<array, shape=(4, 1492, 1491), dtype=float64, chunksize=(4, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int32 16B 1 2 3 4
  * y        (y) float64 12kB 5.927e+06 5.927e+06 ... 5.923e+06 5.923e+06
  * x        (x) float64 12kB 6.626e+05 6.626e+05 ... 6.671e+05 6.671e+05
Attributes: (12/16)
    transform:           (3.0, 0.0, 662619.3139640046, 0.0, -3.0, 5927033.039...
    crs:                 32631
    res:                 (3.0, 3.0)
    is_tiled:            1
    nodatavals:          (0.0, 0.0, 0.0, 0.0)
    _FillValue:          0.0
    ...                  ...
    resampling:          nearest
    AREA_OR_POINT:       Area
    _data_are_separate:  0
    _data_are_stacked:   0
    x_shift_px:          0.10465466820119154
    y_shift_px:          0.3203018290162849


'C:\\development\\coastseg-planet\\20230412_094432_88_2439_3B_coregister_with_combined_mask.tif'

# Convert Planet Imagery to Compatible Format for Model
- reorder to bands to be  RGBNIR
- scale the values to be 0 - 255 

In [7]:
from coastseg_planet.processing import convert_planet_to_model_format

planet_model_img_path = coregistered_planet_path.replace('coregister_with_combined_mask.tif','coregistered_model_ready.tif')

planet_model_img_path= convert_planet_to_model_format(coregistered_planet_path,planet_model_img_path)
planet_model_img_path

red min: 0, red max: 3851
green min: 0, green max: 3654
blue min: 0, blue max: 3837
red min: 0, red max: 255
green min: 0, green max: 255
blue min: 0, blue max: 255


'C:\\development\\coastseg-planet\\20230412_094432_88_2439_3B_coregistered_model_ready.tif'

# Apply the Model to the Planet Imagery

In [8]:
from coastseg_planet.model import apply_model_to_image

apply_model_to_image(planet_model_img_path)

Version:  2.15.0
Eager mode:  True



Some layers from the model checkpoint at nvidia/mit-b0 were not used when initializing TFSegformerForSemanticSegmentation: ['classifier']
- This IS expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFSegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Processing image:  C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregistered_model_ready.tif
Output file:  C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregistered_model_ready_predseg.png
Output file:  C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregistered_model_ready_predseg.png
NC: 4, NDB: 3, TS: [512, 512], TTA: False, OT: False

saving segmentation results to C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregistered_model_ready_predseg.png
writing metadata to C:\development\coastseg-planet\20230412_094432_88_2439_3B_coregistered_model_ready_res.npz


# Extract Shorelines from the Segmentation